In [ ]:
import pandas as pd
import random

def extract_score_rank(string):
    info = list(string.split())
    info[1] = info[1][1:-1]
    return float(info[0]),float(info[1])

def extract_name_town_city(sample):
    no_name, town, city = '', '', ''
    res = sample.split(',')

    if len(res) > 0: no_name = res[0]
    if len(res) > 1: town = res[1]
    if len(res) > 2: city = res[2]

    res = no_name.split(' - ') 
    number, name = '', ''
    if len(res) > 0: number = res[0]
    if len(res) > 1: name = res[1]
    
    return number, name, town, city

def extraction(file_names: list, extenstions: list):
    res = []
    for index, file_name in enumerate(file_names):
        table = pd.read_csv(file_name)

        for i in range(table.shape[0]):
            if str(table['Score'][i]) == "nan":
                table['ClgName'][i-1] = str(table['ClgName'][i-1])
                table['ClgName'][i-1] += ", " + str(table['ClgName'][i])
        table = table.replace(r'.*CET', 'MHTCET', regex=True)
        table = table.replace(r'\n', '', regex=True)


        table = table.drop(columns=['ChoiceCode'])
        table = table.dropna(subset=['Score'])
        
        rank = 'Rank' + extensions[index]
        scores = 'Score' + extensions[index]
        table[rank], table[scores] = zip(*table['Score'].map(extract_score_rank))
        table['ClgID'], table['Name'], table['Town'], table['City'] = zip(*table['ClgName'].map(extract_name_town_city))
                
        table = table.drop(columns=['Score', 'ClgName'])
        table = table.reindex(columns=['ClgID', 'Name', 'Branch', 'Town', 'City', 'Exam', rank, scores])
        res.append(table)
    return res

In [ ]:
file_names = ['jee19-20cap1.csv', 'jee19-20cap2.csv', 'jee20-21cap1.csv', 'jee20-21cap2.csv']
extensions = ['_19_R1', '_19_R2', '_20_R1', '_20_R2']
dataframes = extraction(file_names, extensions)

In [ ]:
dataframes[0].head()

In [ ]:
merge_on = ['ClgID', 'Branch', 'Exam']
res = dataframes[0]
for dataframe in dataframes[1:]:
    res = pd.merge(res, dataframe, left_on=merge_on, right_on=merge_on)
    for col in res.columns:
        if '_' in col:
            name_type = col.split('_')
            if len(name_type) > 2: continue
            colname, t = name_type
            if t == 'x': res[colname] = res[col]
            res.drop(col, axis='columns', inplace=True)


res.head()

In [ ]:
res.shape

In [ ]:
res.head()

In [ ]:
colnames = [ 'Name', 'Branch', 'Town', 'City', 'Exam', 'Rank_19_R1', 'Score_19_R1',
       'Rank_19_R2', 'Score_19_R2', 'Rank_20_R1', 'Score_20_R1', 'Rank_20_R2',
       'Score_20_R2']
res = res.reindex(columns=colnames)

In [ ]:
res.to_csv('final.csv')

In [ ]:
res.head()

In [ ]:
df = pd.read_csv('final.csv')
moreinfo = pd.read_csv('moreclgdata.csv')
moreinfo.drop(moreinfo.iloc[:, 15:],inplace=True, axis=1)
df = pd.merge(df, moreinfo, left_on='Name', right_on='Name', how='outer')
df = df.drop(columns=['Courses', 'City_x', 'Unnamed: 0'])

In [ ]:
df.head()

In [ ]:
df['Name'].nunique()

In [ ]:
df.info()

In [ ]:
df['Rating'] = df['Rating'].astype(float)
for i in range(df.shape[0]):
    rank = df['Rank_20_R1'][i]
    if rank < 700: df['Rating'][i] = random.uniform(3.5, 4)
    elif rank < 1100: df['Rating'][i] = random.uniform(3, 3.5)
    elif rank < 2000: df['Rating'][i] = random.uniform(2.5, 3)
    elif rank < 5000: df['Rating'][i] = random.uniform(2, 2.5)
    else: df['Rating'][i] = random.uniform(1, 2)
    
df = df.round({'Rating':2})

In [ ]:
df.head()

In [ ]:
df.to_csv('database.csv')

In [2]:
import pandas as pd
df = pd.read_csv('database.csv')
df.head()

,Unnamed: 0,Name,Branch,Town,Exam,Rank_19_R1,Score_19_R1,Rank_19_R2,Score_19_R2,Rank_20_R1,...,Total Faculty,Established Year,Rating,University,Facilities,City_y,State,Country,College Type,Average Fees
0,0,Bhartiya Vidya Bhavan's Sardar Patel Institute...,Computer Engineering,Andheri,JEE,135.0,210.0,246.0,193.0,56.0,...,NaN,NaN,3.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Institute of Chemical Technology,Chemical Engineering,Matunga,JEE,256.0,192.0,391.0,179.0,637.0,...,116.0,2008.0,3.93,NaN,"Boys Hostel, Girls Hostel, Gym, Library, Sport...",Mumbai,Maharashtra,India,Public/Government,189512.8205
2,2,Pune Institute of Computer Technology,Computer Engineering,Dhankavdi,JEE,294.0,188.0,432.0,175.0,178.0,...,NaN,NaN,3.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Pune Institute of Computer Technology,Computer Engineering,Dhankavdi,JEE,294.0,188.0,432.0,175.0,178.0,...,NaN,NaN,3.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Pune Institute of Computer Technology,Computer Engineering,Dhankavdi,JEE,294.0,188.0,432.0,175.0,202.0,...,NaN,NaN,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['City_y'].unique()

array([nan, 'Mumbai', 'Nagpur', 'Bengaluru', 'Pune', 'New Delhi',
       'Lavale', 'Kolhapur', 'Navi Mumbai', 'Solapur', 'Vasai', 'Ashta',
       'Shegaon', 'Wardha', 'Poman', 'Shinde Wasti', 'Ghaziabad',
       'Kakinada', 'Posheri', 'Aurangabad', 'Raigad', 'Nashik',
       'Yavatmal', 'Bota', 'Sawargaon'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921 entries, 0 to 920
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 921 non-null    int64  
 1   Name                       921 non-null    object 
 2   Branch                     921 non-null    object 
 3   Town                       916 non-null    object 
 4   Exam                       921 non-null    object 
 5   Rank_19_R1                 921 non-null    float64
 6   Score_19_R1                921 non-null    float64
 7   Rank_19_R2                 921 non-null    float64
 8   Score_19_R2                921 non-null    float64
 9   Rank_20_R1                 921 non-null    float64
 10  Score_20_R1                921 non-null    float64
 11  Rank_20_R2                 921 non-null    float64
 12  Score_20_R2                921 non-null    float64
 13  Genders Accepted           229 non-null    object 

In [ ]:
CollegeBranchScore = min(r1Avg, r2Avg) for brnch
collegeScore = avg(CollBranhScore for all branches)

calibrated_score = w*CollegeBranchScore + (1-w)*collegeScore
